Check completeness of the tables

In [0]:
source_schema = "samples.tpch"
target_schema = "sandwich.bronze"

tables = [row.tableName for row in spark.sql(f"SHOW TABLES IN {source_schema}").collect()]

for tbl in tables:
    src_count = spark.sql(f"SELECT COUNT(*) as cnt FROM {source_schema}.{tbl}").collect()[0]['cnt']
    dst_count = spark.sql(f"SELECT COUNT(*) as cnt FROM {target_schema}.{tbl}").collect()[0]['cnt']
    
    if src_count == dst_count:
        print(f"[OK] Table {tbl}: row count matches ({src_count})")
    else:
        print(f"[ERROR] Table {tbl}: source={src_count}, bronze={dst_count}")


Check accuracy of the data in the tables by hashing

In [0]:
import pyspark.sql.functions as F

for tbl in tables:
    src_sum = spark.table(f"{source_schema}.{tbl}").agg(F.sum(F.hash("*"))).collect()[0][0]
    dst_sum = spark.table(f"{target_schema}.{tbl}").agg(F.sum(F.hash("*"))).collect()[0][0]
    if src_sum == dst_sum:
        print(f"{tbl} checksum OK")
    else:
        print(f"{tbl} checksum MISMATCH")
